In [ ]:
from utils import constants_sql, helpers

In [ ]:
logger = helpers.setup_logger_stdout('geocode')

In [ ]:
conn = helpers.get_database_connection()
cur = conn.cursor()
dict_cur = helpers.get_dictionary_cursor(conn)

In [ ]:
results = helpers.get_records2geocode(dict_cur, logger)
number_results = len(results)
logger.info(f"{number_results} records to GeoCode")

In [ ]:
geocode_updates = []

In [ ]:
%%time
for row in results:
    geocode_results = helpers.extract_geocodes(row, logger)
    if geocode_results:
        geocode_updates.append(geocode_results)

In [ ]:
len(geocode_updates)

In [ ]:
geocode_updates[0]

In [ ]:
%%time
cur.executemany(constants_sql.UPDATE_GEOCODING_SQL, geocode_updates)
conn.commit()

In [ ]:
conn.close()
cur.close()
dict_cur.close()